<a href="https://colab.research.google.com/github/ohadbarr1/Thesis/blob/main/Chronos_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations and initializations

Install chronos

In [1]:
pip install git+https://github.com/amazon-science/chronos-forecasting.git

  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-req-build-ntje5icy
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-req-build-ntje5icy
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit eb7bdfc047de3e7af972b4ee7cf23a7968b7daa3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for chronos: filename=chronos-1.2.0-py3-none-any.whl size=13908 sha256=a78965f69811f3fc0dff9641d8179a5704ce328475bc6076276667240cb452e8
  Stored in directory: /tmp/pip-ephem-wheel-cache-9dwyvjqd/wheels/bf/c1/65/08857e57345ef1b34ba0edb4791f1b2594943e82f34e93a4ab
Successfully built chronos


## **Import libraries**

In [2]:
# Import relevant libraries
import pandas as pd
import numpy as np
from google.colab import files
from datetime import timedelta

# Data preprocessing
from sklearn.preprocessing import MinMaxScaler

# Gather financial data
import yfinance as yf

# Plots
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Models
import torch
from sklearn.model_selection import train_test_split
from chronos import ChronosPipeline

## Initializing pipeline

In [3]:
# Use the Chronos-t5-base model, 200m parameters
pipeline = ChronosPipeline.from_pretrained(
  "amazon/chronos-t5-base",
  device_map="cuda",
  torch_dtype=torch.bfloat16,
)

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/806M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

### Gather VIX Data

In [4]:
# Data fetching and preprocessing
tickers = ['^VIX9D', '^VIX', '^VIX3M', '^VIX6M']
data = yf.download(tickers, start="2011-01-03")['Close']
# Reordering columns and renaming them according to your format
data = data[['^VIX', '^VIX3M', '^VIX6M', '^VIX9D']]
data.columns = ['VIX', 'VIX3M', 'VIX6M', 'VIX9D']
# Resetting index to make 'Date' a column
data.reset_index(inplace=True)

data



[*********************100%***********************]  4 of 4 completed


,Date,VIX,VIX3M,VIX6M,VIX9D
0,2011-01-03 00:00:00+00:00,17.610001,20.620001,23.400000,16.040001
1,2011-01-04 00:00:00+00:00,17.379999,20.610001,23.190001,16.059999
2,2011-01-05 00:00:00+00:00,17.020000,20.049999,22.780001,15.570000
3,2011-01-06 00:00:00+00:00,17.400000,20.350000,22.870001,15.710000
4,2011-01-07 00:00:00+00:00,17.139999,20.290001,22.920000,15.010000
...,...,...,...,...,...
3433,2024-08-26 00:00:00+00:00,16.150000,18.250000,19.360001,14.870000
3434,2024-08-27 00:00:00+00:00,15.430000,17.730000,19.000000,14.170000
3435,2024-08-28 00:00:00+00:00,17.110001,18.900000,19.830000,16.639999
3436,2024-08-29 00:00:00+00:00,15.650000,18.000000,19.309999,13.730000


In [6]:
VIX9D = data[['Date','VIX9D']]
VIX = data[['Date','VIX']]
VIX3M = data[['Date','VIX3M']]
VIX6M = data[['Date','VIX6M']]


VIX Data

In [ ]:
# Avoid SettingWithCopyWarning by using .loc
VIX.loc[:, 'Date'] = pd.to_datetime(VIX['Date'])

# Initialize a DataFrame to store results
results = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])

# Number of days in three days (approx.)
days = timedelta(days=2)

# Loop over the DataFrame
for i in range(872, len(VIX),2):
    # Define the current date
    current_date = VIX['Date'].iloc[i]
    end_date = current_date + days

    # Use all data up to the current date as context
    context_data = VIX[VIX['Date'] <= current_date]

    # Filter data for the next three days (actual future values)
    actual_next_day = VIX[(VIX['Date'] > current_date) & (VIX['Date'] <= end_date)]

    # Convert the entire context up to the current date to a tensor
    context = torch.tensor(context_data["VIX"].values, dtype=torch.float32).unsqueeze(0)

    # Initialize a list to store forecasts
    forecasts = []

    # Make a prediction for the next three days
    prediction_length = min(len(actual_next_day), 1)  # Predicting 2 days or fewer if the actual data is less
    if prediction_length > 0:  # Ensure there's something to predict
        forecast = pipeline.predict(context, prediction_length)
        if forecast is not None and len(forecast) > 0:
            forecast = forecast.squeeze().cpu().numpy()

            # Ensure forecast is 1-dimensional
            if forecast.ndim > 1:
                forecast = forecast.flatten()

            # Store the forecast
            forecasts.append(forecast)

    # Proceed only if we have forecasts
    if forecasts:
        # Ensure the lengths of actual and forecasts match
        forecast_dates = actual_next_day['Date'].values
        aligned_length = min(len(forecast_dates), len(forecasts[0]))
        forecast_dates = forecast_dates[:aligned_length]
        actual_values = actual_next_day['VIX'].values[:aligned_length]

        # Truncate forecasts to the aligned length
        forecasts = [forecast[:aligned_length] for forecast in forecasts]

        # Calculate the average of the 5 forecasts
        forecast_avg = np.mean(forecasts, axis=0)

        # Store the results in the DataFrame
        result_df = pd.DataFrame({
            'Date': forecast_dates,
            'Actual': actual_values,
            'Forecast_Avg': forecasts[0]
        })

        # Avoid concatenating empty dataframes
        if not result_df.empty:
            # Append to the main results DataFrame
            results = pd.concat([results, result_df], ignore_index=True)

# Reset index of the final DataFrame
results.reset_index(drop=True, inplace=True)

# Create the interactive plot
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add the actual values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Values',
    line=dict(color='blue')
))

# Add the averaged forecast values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Forecast_Avg'],
    mode='lines',
    name='Forecasted Values (Avg)',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title="Forecasted vs Actual VIX Values (Next-Day Predictions)",
    xaxis_title="Date",
    yaxis_title="VIX Value",
    legend_title="Legend",
    hovermode="x unified"
)

# Show the interactive plot
fig.show()

# prompt: convert to csv and download results_df and call it "VIX.csv"

# Assuming 'results' is your DataFrame
results.to_csv('Chronos_VIX_pred.csv', index=False)
files.download('Chronos_VIX_pred.csv')


<ipython-input-28-603cf525c8ec>:66: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



VIX 9D

In [10]:
# Avoid SettingWithCopyWarning by using .loc
VIX9D.loc[:, 'Date'] = pd.to_datetime(VIX9D['Date'])

# Initialize a DataFrame to store results
results = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])

# Number of days in three days (approx.)
days = timedelta(days=2)

# Loop over the DataFrame
for i in range(872, len(VIX9D), 2):
    # Define the current date
    current_date = VIX9D['Date'].iloc[i]
    end_date = current_date + days

    # Use all data up to the current date as context
    context_data = VIX9D[VIX9D['Date'] <= current_date]

    # Filter data for the next three days (actual future values)
    actual_next_day = VIX9D[(VIX9D['Date'] > current_date) & (VIX9D['Date'] <= end_date)]

    # Convert the entire context up to the current date to a tensor
    context = torch.tensor(context_data["VIX9D"].values, dtype=torch.float32).unsqueeze(0)

    # Initialize a list to store forecasts
    forecasts = []

    # Make a prediction for the next three days
    prediction_length = min(len(actual_next_day), 1)  # Predicting 1 day or fewer if the actual data is less
    if prediction_length > 0:  # Ensure there's something to predict
        forecast = pipeline.predict(context, prediction_length)
        if forecast is not None and len(forecast) > 0:
            forecast = forecast.squeeze().cpu().numpy()

            # Ensure forecast is 1-dimensional
            if forecast.ndim > 1:
                forecast = forecast.flatten()

            # Store the forecast
            forecasts.append(forecast)

    # Proceed only if we have forecasts
    if forecasts:
        # Ensure the lengths of actual and forecasts match
        forecast_dates = actual_next_day['Date'].values
        aligned_length = min(len(forecast_dates), len(forecasts[0]))
        forecast_dates = forecast_dates[:aligned_length]
        actual_values = actual_next_day['VIX9D'].values[:aligned_length]

        # Truncate forecasts to the aligned length
        forecasts = [forecast[:aligned_length] for forecast in forecasts]

        # Calculate the average of the 5 forecasts
        forecast_avg = np.mean(forecasts, axis=0)

        # Store the results in the DataFrame
        result_df = pd.DataFrame({
            'Date': forecast_dates,
            'Actual': actual_values,
            'Forecast_Avg': forecasts[0]
        })

        # Avoid concatenating empty dataframes
        if not result_df.empty:
            # Append to the main results DataFrame
            results = pd.concat([results, result_df], ignore_index=True)

# Reset index of the final DataFrame
results.reset_index(drop=True, inplace=True)

# Create the interactive plot
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add the actual values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Values',
    line=dict(color='blue')
))

# Add the averaged forecast values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Forecast_Avg'],
    mode='lines',
    name='Forecasted Values (Avg)',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title="Forecasted vs Actual VIX9D Values (Next-Day Predictions)",
    xaxis_title="Date",
    yaxis_title="VIX9D Value",
    legend_title="Legend",
    hovermode="x unified"
)

# Show the interactive plot
fig.show()


# Assuming 'results' is your DataFrame
results.to_csv('Chronos_VIX9D_pred.csv', index=False)
files.download('Chronos_VIX9D_pred.csv')


<ipython-input-10-a281b9667b2e>:66: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

VIX3M

In [11]:
# Avoid SettingWithCopyWarning by using .loc
VIX3M.loc[:, 'Date'] = pd.to_datetime(VIX3M['Date'])

# Initialize a DataFrame to store results
results = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])

# Number of days in three days (approx.)
day = timedelta(days=1)

# Loop over the DataFrame
for i in range(872, len(VIX3M), 2):
    # Define the current date
    current_date = VIX3M['Date'].iloc[i]
    end_date = current_date + day

    # Use all data up to the current date as context
    context_data = VIX3M[VIX3M['Date'] <= current_date]

    # Filter data for the next three days (actual future values)
    actual_next_day = VIX3M[(VIX3M['Date'] > current_date) & (VIX3M['Date'] <= end_date)]

    # Convert the entire context up to the current date to a tensor
    context = torch.tensor(context_data["VIX3M"].values, dtype=torch.float32).unsqueeze(0)

    # Initialize a list to store forecasts
    forecasts = []

    # Make a prediction for the next three days
    prediction_length = min(len(actual_next_day), 1)  # Predicting 3 days or fewer if the actual data is less
    if prediction_length > 0:  # Ensure there's something to predict
        forecast = pipeline.predict(context, prediction_length)
        if forecast is not None and len(forecast) > 0:
            forecast = forecast.squeeze().cpu().numpy()

            # Ensure forecast is 1-dimensional
            if forecast.ndim > 1:
                forecast = forecast.flatten()

            # Store the forecast
            forecasts.append(forecast)

    # Proceed only if we have forecasts
    if forecasts:
        # Ensure the lengths of actual and forecasts match
        forecast_dates = actual_next_day['Date'].values
        aligned_length = min(len(forecast_dates), len(forecasts[0]))
        forecast_dates = forecast_dates[:aligned_length]
        actual_values = actual_next_day['VIX3M'].values[:aligned_length]

        # Truncate forecasts to the aligned length
        forecasts = [forecast[:aligned_length] for forecast in forecasts]

        # Calculate the average of the 5 forecasts
        forecast_avg = np.mean(forecasts, axis=0)

        # Store the results in the DataFrame
        result_df = pd.DataFrame({
            'Date': forecast_dates,
            'Actual': actual_values,
            'Forecast_Avg': forecasts[0]
        })

        # Avoid concatenating empty dataframes
        if not result_df.empty:
            # Append to the main results DataFrame
            results = pd.concat([results, result_df], ignore_index=True)

# Reset index of the final DataFrame
results.reset_index(drop=True, inplace=True)

# Create the interactive plot
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add the actual values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Values',
    line=dict(color='blue')
))

# Add the averaged forecast values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Forecast_Avg'],
    mode='lines',
    name='Forecasted Values (Avg)',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title="Forecasted vs Actual VIX3M Values (Next-Day Predictions)",
    xaxis_title="Date",
    yaxis_title="VIX3M Value",
    legend_title="Legend",
    hovermode="x unified"
)

# Show the interactive plot
fig.show()


# Assuming 'results' is your DataFrame
results.to_csv('Chronos_VIX3M_pred.csv', index=False)
files.download('Chronos_VIX3M_pred.csv')


<ipython-input-11-784ea701a69c>:66: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

VIX6M

In [13]:
# Avoid SettingWithCopyWarning by using .loc
VIX6M.loc[:, 'Date'] = pd.to_datetime(VIX6M['Date'])

# Initialize a DataFrame to store results
results = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])

# Number of days in two days (approx.)
days = timedelta(days=2)

# Loop over the DataFrame
for i in range(872, len(VIX6M), 2):
    # Define the current date
    current_date = VIX6M['Date'].iloc[i]
    end_date = current_date + days

    # Use all data up to the current date as context
    context_data = VIX6M[VIX6M['Date'] <= current_date]

    # Filter data for the next three days (actual future values)
    actual_next_day = VIX6M[(VIX6M['Date'] > current_date) & (VIX6M['Date'] <= end_date)]

    # Convert the entire context up to the current date to a tensor
    context = torch.tensor(context_data["VIX6M"].values, dtype=torch.float32).unsqueeze(0)

    # Initialize a list to store forecasts
    forecasts = []

    # Make a prediction for the next three days
    prediction_length = min(len(actual_next_day), 1)  # Predicting 3 days or fewer if the actual data is less
    if prediction_length > 0:  # Ensure there's something to predict
        forecast = pipeline.predict(context, prediction_length)
        if forecast is not None and len(forecast) > 0:
            forecast = forecast.squeeze().cpu().numpy()

            # Ensure forecast is 1-dimensional
            if forecast.ndim > 1:
                forecast = forecast.flatten()

            # Store the forecast
            forecasts.append(forecast)

    # Proceed only if we have forecasts
    if forecasts:
        # Ensure the lengths of actual and forecasts match
        forecast_dates = actual_next_day['Date'].values
        aligned_length = min(len(forecast_dates), len(forecasts[0]))
        forecast_dates = forecast_dates[:aligned_length]
        actual_values = actual_next_day['VIX6M'].values[:aligned_length]

        # Truncate forecasts to the aligned length
        forecasts = [forecast[:aligned_length] for forecast in forecasts]

        # Calculate the average of the 5 forecasts
        forecast_avg = np.mean(forecasts, axis=0)

        # Store the results in the DataFrame
        result_df = pd.DataFrame({
            'Date': forecast_dates,
            'Actual': actual_values,
            'Forecast_Avg': forecasts[0]
        })

        # Avoid concatenating empty dataframes
        if not result_df.empty:
            # Append to the main results DataFrame
            results = pd.concat([results, result_df], ignore_index=True)

# Reset index of the final DataFrame
results.reset_index(drop=True, inplace=True)

# Create the interactive plot
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add the actual values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Values',
    line=dict(color='blue')
))

# Add the averaged forecast values line
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Forecast_Avg'],
    mode='lines',
    name='Forecasted Values (Avg)',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title="Forecasted vs Actual VIX6M Values (Next-Day Predictions)",
    xaxis_title="Date",
    yaxis_title="VIX6M Value",
    legend_title="Legend",
    hovermode="x unified"
)

# Show the interactive plot
fig.show()


# Assuming 'results' is your DataFrame
results.to_csv('Chronos_VIX6M_pred.csv', index=False)
files.download('Chronos_VIX6M_pred.csv')


<ipython-input-13-69cd3347fdac>:66: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Signal Predictions

In [18]:
def classify_row(row):
    if row['VIX9D'] < row['VIX'] and row['VIX'] < row['VIX3M']:
        if row['VIX9D'] < 9:
            return 0
        else:
            return 1
    elif row['VIX9D'] - row['VIX'] > 1 and row['VIX9D'] - row['VIX3M'] > 1 and row['VIX9D'] - row['VIX6M'] > 1 :
        if row['VIX9D'] > 40:
            return 1
        else:
            return -1
    elif row['VIX9D'] - row['VIX'] > 1 and row['VIX9D'] < row['VIX3M']:
        return 0
    else:
        return 1

data["Signal"] = data.apply(classify_row, axis=1)
Signal = data[["Date", "Signal"]]

In [24]:
# Fill in missing dates in the Signal DataFrame
all_dates = pd.date_range(start=Signal['Date'].min(), end=Signal['Date'].max(), freq='D')
Signal = Signal.set_index('Date').reindex(all_dates).reset_index().rename(columns={'index': 'Date'})

# Ensure 'Date' column exists after reindexing
print("After reindexing Signal:", Signal.columns)

# Initialize the results DataFrame
results = pd.DataFrame(columns=['Date', 'Actual', 'Forecast_signal'])

# Number of days in one day
day = timedelta(days=1)

# Loop over the DataFrame
for i in range(872, len(Signal)):
    current_date = Signal['Date'].iloc[i]
    end_date = current_date + day

    context_data = Signal[Signal['Date'] <= current_date]
    actual_next_day = Signal[(Signal['Date'] > current_date) & (Signal['Date'] <= end_date)]

    # Convert the entire context up to the current date to a tensor
    context = torch.tensor(context_data["Signal"].values, dtype=torch.float32).unsqueeze(0)

    forecasts = []
    prediction_length = min(len(actual_next_day), 1)  # Predicting 1 day

    if prediction_length > 0:
        forecast = pipeline.predict(context, prediction_length)
        if forecast is not None and len(forecast) > 0:
            forecast = forecast.squeeze().cpu().numpy()
            if forecast.ndim > 1:
                forecast = forecast.flatten()
            forecasts.append(forecast)

    if forecasts:
        forecast_dates = actual_next_day['Date'].values
        aligned_length = min(len(forecast_dates), len(forecasts[0]))

        if aligned_length > 0:
            forecast_dates = forecast_dates[:aligned_length]
            actual_values = actual_next_day['Signal'].values[:aligned_length]
            forecasts = [forecast[:aligned_length] for forecast in forecasts]

            result_df = pd.DataFrame({
                'Date': forecast_dates,
                'Actual': actual_values,
                'Forecast_signal': forecasts[0]
            })

            # Debugging: Check the result_df before concatenation
            print(f"Result DataFrame at iteration {i}:")
            print(result_df.head())

            if not result_df.empty and result_df.notna().all().all():
                results = pd.concat([results, result_df], ignore_index=True)

# Reset index of the final DataFrame
results.reset_index(drop=True, inplace=True)

# Debugging: Check the results DataFrame before reindexing
print("Before reindexing results DataFrame:", results.columns)

# Ensure all dates are in the results DataFrame
results = results.set_index('Date').reindex(all_dates).reset_index()

# Handle NA values in 'Actual' and 'Forecast_signal' columns
results['Actual'] = results['Actual'].ffill()  # Forward fill for actual values
results['Forecast_signal'] = results['Forecast_signal'].fillna(0)  # Fill forecast NaNs with 0 or a placeholder value

# Final debug output before plotting
print("Final results DataFrame:", results.head())

# Create the interactive plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Actual'],
    mode='lines',
    name='Actual Values',
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=results['Date'],
    y=results['Forecast_signal'],
    mode='lines',
    name='Forecasted Values',
    line=dict(color='red')
))

fig.update_layout(
    title="Forecasted vs Actual Signal Values (Next-Day Predictions)",
    xaxis_title="Date",
    yaxis_title="Signal Value",
    legend_title="Legend",
    hovermode="x unified"
)

fig.show()

# Save the results to a CSV file
results.to_csv('Chronos_Signal_pred.csv', index=False)
files.download('Chronos_Signal_pred.csv')

After reindexing Signal: Index(['Date', 'Signal'], dtype='object')
Result DataFrame at iteration 872:
        Date  Actual  Forecast_signal
0 2013-05-25     NaN         0.998738
Result DataFrame at iteration 873:
        Date  Actual  Forecast_signal
0 2013-05-26     NaN          0.99868
Result DataFrame at iteration 874:
        Date  Actual  Forecast_signal
0 2013-05-27     NaN     1.369044e-08
Result DataFrame at iteration 875:
        Date  Actual  Forecast_signal
0 2013-05-28     0.0          0.99868


<ipython-input-24-b7daa42df9d3>:56: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Streaming output truncated to the last 5000 lines.
0 2020-02-09     NaN         1.003083
Result DataFrame at iteration 3324:
        Date  Actual  Forecast_signal
0 2020-02-10     1.0         1.003083
Result DataFrame at iteration 3325:
        Date  Actual  Forecast_signal
0 2020-02-11     1.0         1.003107
Result DataFrame at iteration 3326:
        Date  Actual  Forecast_signal
0 2020-02-12     1.0         1.003107
Result DataFrame at iteration 3327:
        Date  Actual  Forecast_signal
0 2020-02-13     1.0         1.003107
Result DataFrame at iteration 3328:
        Date  Actual  Forecast_signal
0 2020-02-14     1.0         1.003107
Result DataFrame at iteration 3329:
        Date  Actual  Forecast_signal
0 2020-02-15     NaN         1.003107
Result DataFrame at iteration 3330:
        Date  Actual  Forecast_signal
0 2020-02-16     NaN         1.003083
Result DataFrame at iteration 3331:
        Date  Actual  Forecast_signal
0 2020-02-17     NaN         1.003083
Result DataFram

KeyError: 'Date'

In [25]:
results

,index,Actual,Forecast_signal
0,2011-01-03 00:00:00+00:00,NaN,0.0
1,2011-01-04 00:00:00+00:00,NaN,0.0
2,2011-01-05 00:00:00+00:00,NaN,0.0
3,2011-01-06 00:00:00+00:00,NaN,0.0
4,2011-01-07 00:00:00+00:00,NaN,0.0
...,...,...,...
4984,2024-08-26 00:00:00+00:00,NaN,0.0
4985,2024-08-27 00:00:00+00:00,NaN,0.0
4986,2024-08-28 00:00:00+00:00,NaN,0.0
4987,2024-08-29 00:00:00+00:00,NaN,0.0
